In [1]:
#pip install pillow
#pip install pytesseract
#pip install wand
from PIL import Image
import pytesseract
import pandas as pd
import io
from wand.image import Image as wi

pd.set_option('max_colwidth', 2000)
pd.options.display.max_rows = 500
import pandas as pd

#First install tesseract-ocr from this link https://github.com/UB-Mannheim/tesseract/wiki
#Then add this line for tesseract ocr to work
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe"

#### Downloading the PDF from washington post 

In [2]:
#https://www.washingtonpost.com/context/read-the-whistleblower-complaint-regarding-president-trump-s-communications-with-ukrainian-president-volodymyr-zelensky/4b9e0ca5-3824-467f-b1a3-77f2d4ee16aa/
from urllib.request import Request, urlopen 
import requests  
url="https://games-cdn.washingtonpost.com/notes/prod/default/documents/3b5487de-f987-4cef-b59b-c29bb67687ac/note/ef3465c1-465b-4e68-9b36-08b5946b0df4.pdf#page=1"  
import urllib.request  
req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})  
r = requests.get(url)

with open("trump_ukrain.pdf", "wb") as code:
    code.write(r.content)

#### Extract the text from the pdf file downloaded

In [3]:
pdf = wi(filename = "trump_ukrain.pdf", resolution = 300)
pdfImg = pdf.convert('jpeg')

imgBlobs = []

for img in pdfImg.sequence:
    page = wi(image = img)
    imgBlobs.append(page.make_blob('jpeg'))

extracted_text = []

for imgBlob in imgBlobs:
    im = Image.open(io.BytesIO(imgBlob))
    text = pytesseract.image_to_string(im, lang = 'eng')
    extracted_text.append(text)

print('Done extracting text')

Done extracting text


#### Save the extracted text to a dataframe

In [4]:
df = pd.DataFrame(extracted_text,columns =['Page_Text'])\
.replace(r'\n',' ', regex=True)
df

,Page_Text
0,"UNCLASSIFIED August 12, 2019 The Honorable Richard Burr Chairman Select Committee on Intelligence United States Senate The Honorable Adam Schiff Chairman Permanent Select Committee on Intelligence United States House of Representatives Dear Chairman Burr and Chairman Schiff: I am reporting an “urgent concern” in accordance with the procedures outlined in 50 U.S.C. §3033(k)(5)(A). This letter is UNCLASSIFIED when separated from the attachment. In the course of my official duties, I have received information from multiple U.S. ' Government officials that the President of the United States is using the power of his office to solicit interference from a foreign country in the 2020 U.S. election. This interference includes, among other things, pressuring a foreign country to investigate one of the President’s main domestic political rivals. The President’s personal lawyer, Mr. Rudolph Giuliani, is a central figure in this effort. Attorney General Barr appears to be involved as well. e Over the past four months, more than half a dozen U.S. officials have informed me of various facts related to this effort. The information provided herein was relayed to me in the course of official interagency business. It is routine for U.S. officials with responsibility for a particular regional or functional portfolio to share such information with one another in order to inform policymaking and analysis. e J was nota direct witness to most of the events described. However, I found my colleagues’ accounts of these events to be credible because, in almost all cases, multiple officials recounted fact patterns that were consistent with one another. In addition, a variety of information consistent with these private accounts has been reported publicly. I am deeply concerned that the actions described below constitute “‘a serious or flagrant problem, abuse, or violation of law or Executive Order” that “does not include differences of opinions concerning public policy matters..."
1,"UNCLASSIFIED To the best of my knowledge, the entirety of this statement is unclassified when separated from the classified enclosure. I have endeavored to apply the classification standards outlined in Executive Order (EO) 13526 and to separate out information that I know or have reason to believe is classified for national security purposes. ! e Ifaclassification marking is applied retroactively, I believe it is incumbent upon the classifying authority to explain why such a marking was applied, and to which specific information it pertains. I. The 25 July Presidential phone call Early in the morning of 25 July, the President spoke by telephone with Ukrainian President Volodymyr Zelenskyy. I do not know which side initiated the call. This was the first publicly acknowledged call between the two leaders since a brief congratulatory call after Mr. Zelenskyy won the presidency on 21 April. Multiple White House officials with direct knowledge of the call informed me that, after an initial exchange of pleasantries, the President used the remainder of the call to advance his personal interests. Namely, he sought to pressure the Ukrainian leader to take actions to help the President’s 2020 reelection bid. According to the White House officials who had direct knowledge of the call, the President pressured Mr. Zelenskyy to, inter alia: e initiate or continue an investigation? into the activities of former Vice President Joseph Biden and his son, Hunter Biden; e assist in purportedly uncovering that allegations of Russian interference in the 2016 U.S. presidential election originated in Ukraine, with a specific request that the Ukrainian leader locate and turn over servers used by the Democratic National Committee (DNC) and examined by the U.S. cyber security firm Crowdstrike,? which initially reported that Russian hackers had penetrated the DNC’s networks in 2016; and e meet or speak with two people the President named explicitly as his personal envoys on thes..."
2,"U

#### Named Entity Recognition

In [5]:
#Import spacy to extract the data from the text file
import spacy
import pandas as pd
pd.set_option('max_colwidth', 2000)
pd.options.display.max_rows = 500

nlp = spacy.load('en_core_web_lg')

In [6]:
# Read the text file
doc = nlp(df['Page_Text'].to_string())

In [7]:
# Get the entities, labels and explaination to the labels
table = []
for ent in doc.ents:
    table.append([ent.text,ent.label_,spacy.explain(ent.label_)])
    
# Creat a dataframe from the list created above
df2 = pd.DataFrame(table, columns=['Entity', 'Label','Label_Description']).sort_values(by=['Label'])
print(df2.shape)

#filter for Date, org, person, time and work_of_art and dedupe
df3= df2[df2['Label'].isin(['DATE','ORG','PERSON','TIME','WORK_OF_ART'])]
print(df3.shape)
df4=df3.drop_duplicates(subset=None, keep='first',inplace= False)
print(df4.shape)
df4

(308, 3)
(192, 3)
(110, 3)


,Entity,Label,Label_Description
188,25 April,DATE,Absolute or relative dates or periods
147,2016,DATE,Absolute or relative dates or periods
208,9 May,DATE,Absolute or relative dates or periods
206,14 May,DATE,Absolute or relative dates or periods
102,about 2 August,DATE,Absolute or relative dates or periods
304,early August,DATE,Absolute or relative dates or periods
303,26 July,DATE,Absolute or relative dates or periods
53,March 2019,DATE,Absolute or relative dates or periods
302,23 July,DATE,Absolute or relative dates or periods
191,about 29 April,DATE,Absolute or relative dates or periods


In [8]:
# Display the entities and their Labels in the actual document
spacy.displacy.render(doc,style ='ent', jupyter = True)